In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
 
# Load dataset
df = pd.read_csv("smaller_dataset.csv")
 
# Preprocess the dataset
df_encoded = df.drop(columns=[df.columns[0]])  # Drop the first column
X = df_encoded.drop(columns=["DRUH_POVR", "NAZ_LOKALI"])  # Features
y = df_encoded["DRUH_POVR"]  # Target
 
# Convert features to TensorFlow tensors
X_tensor = tf.convert_to_tensor(X.to_numpy(), dtype=tf.float32)
 
# Use TensorFlow StringLookup for label encoding
label_lookup = tf.keras.layers.StringLookup(output_mode='int', vocabulary=tf.constant(y.unique()))
y_tensor = label_lookup(y) - 1  # Adjust labels to start from 0
 
# Create a TensorFlow Dataset
num_classes = len(label_lookup.get_vocabulary()) - 1  # Adjust for 0-based indexing
y_one_hot = tf.one_hot(y_tensor, depth=num_classes)
dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_one_hot))
 
# Shuffle the dataset
dataset_size = len(y_tensor)
dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)
 
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size
 
train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)
 
# Batch the datasets for training, validation, and evaluation
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
 
# Determine the number of classes for the output layer
# num_classes = len(label_lookup.get_vocabulary()) - 1  # Adjust for 0-based indexing
y_encoded = y_tensor.numpy()
y_categorical = to_categorical(y_encoded, num_classes=num_classes)
#  Define the model
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),  # Specify input shape
#     tf.keras.layers.Dense(num_classes, activation='softmax'),  # Output layer
# ])
 
 
 
# Compile the model
# model.compile(
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
#     optimizer=tf.keras.optimizers.Adam(),
#     metrics=['accuracy']  # Add accuracy as a metric
# )
 
input_shape=(*X.shape[1:],1)
filters=[1024,512,256,128]
kernel_sizes=[9 for i in range(4)]
padding='causal'
print(y.shape)
model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Input(
        shape=input_shape
    )
)
model.add(
    tf.keras.layers.BatchNormalization()
)
 
for f,k in zip(filters, kernel_sizes):
    model.add(
        tf.keras.layers.Conv1D(
            filters=f,
            kernel_size=k,
            activation=tf.nn.relu,
            padding=padding,
            kernel_regularizer='l2'
        )
    )
    model.add(
        tf.keras.layers.BatchNormalization()
    )
    model.add(tf.keras.layers.ReLU())
 
model.add(tf.keras.layers.GlobalAveragePooling1D())        
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)
 
# Train the model with a validation set
history = model.fit(
    train_dataset, 
    validation_data=val_dataset,  # Include validation set
    epochs=10,  # Increased epochs for better training
    verbose=1  # Display progress
)
 
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

2025-02-18 07:42:38.468338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:38.468459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:38.468615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

(32950,)
Epoch 1/10


2025-02-18 07:42:40.293061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2025-02-18 07:42:41.197721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-02-18 07:42:41.276796: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x776a14400ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-18 07:42:41.276834: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-02-18 07:42:41.289573: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-18 07:42:41.422865: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of th

 54/721 [=>............................] - ETA: 35s - loss: 6.6332 - categorical_accuracy: 0.3218     

KeyboardInterrupt: 

In [11]:
y.shape

(32950,)

In [15]:
len(y.unique())

18

In [2]:
train_dataset.

AttributeError: '_BatchDataset' object has no attribute 'shape'

In [1]:
import tensorflow as tf

# List all physical devices of type 'GPU'
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available:", gpus)
else:
    print("No GPU found. Using CPU.")

2025-02-18 07:42:30.536476: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 07:42:30.571586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-18 07:42:31.133608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPUs are available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-02-18 07:42:31.669760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:31.751669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:31.751827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf